In [10]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
from langdetect import detect

In [11]:
labelledkaggle = pd.read_csv('./only_english_merged_data.csv')

In [4]:
with_similar_tracks = pd.read_csv('./278k_labelled_uri_w_similar.csv')

## Genius and Spotify credentials

In [10]:
genius_key = 'put-your-genius-key'
cid = 'put-your-cid'
secret = 'put-your-secred'

In [11]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [13]:
def get_track_artist_and_song_from_uri(uri):
    split_uri = uri.split(':')
    if len(split_uri) >= 3 and split_uri[1] == 'track':
        track_id = split_uri[2]
        track_info = sp.track(track_id)
        artist_name = track_info['artists'][0]['name']
        track_name = track_info['name']
        return artist_name, track_name
    else:
        return None, None




In [14]:
def request_song_info(track_name, track_artist, genius_key):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': f'Bearer {genius_key}'}
    search_url = f'{base_url}/search'
    data = {'q': f'{track_artist} {track_name}'}
    response = requests.get(search_url, headers=headers, params=data)
    return response

def get_song_info_from_response(response, artist):
    json = response.json()
    remote_song_info = None
    for hit in json['response']['hits']:
        if artist.lower() in hit['result']['artist_names'].lower():
            remote_song_info = hit
            break
    return remote_song_info

def get_url(song_info):
    song_url = song_info['result']['url']
    return song_url

def scrape_lyrics(song_lyrics_url):
    page = requests.get(song_lyrics_url)
    html = BeautifulSoup(page.content, 'html.parser')
    lyrics1 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-5 Dzxov")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics


def get_all_tracks_artist_song_and_lyrics_from_uri(df):
    artists_song_lyrics = []
    for index, row in df[0:278000].iterrows():
        if index%100 == 0:
            df.to_csv('./only_english_merged_data.csv')
        uri = row['uri']
        artist_name, track_name = get_track_artist_and_song_from_uri(uri)
        if artist_name and track_name:
            lyrics = None
            response = request_song_info(artist_name, track_name, genius_key)
            song_info = get_song_info_from_response(response, artist_name)
            if (song_info):
                song_lyrics_url = get_url(song_info)
                lyrics = scrape_lyrics(song_lyrics_url) 
                df['artist'][index] = artist_name
                df['track'][index] = track_name
                df['lyrics'][index] = lyrics
                artists_song_lyrics.append((artist_name, track_name, lyrics))
        else:
            print(f"Row {index + 1}: Invalid URI or not a track URI.")
    return artists_song_lyrics, df

artists_song_lyrics, new_kaggle = get_all_tracks_artist_song_and_lyrics_from_uri(labelledkaggle)

180000


<ipython-input-14-39d49b402d5e>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['artist'][index] = artist_name
<ipython-input-14-39d49b402d5e>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['track'][index] = track_name
<ipython-input-14-39d49b402d5e>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lyrics'][index] = lyrics


180100
180200
180300
180400
180500
180600
180700
180800
180900
Row 180927: Invalid URI or not a track URI.
181000
181100
181200
181300
181400
181500
181600
181700
181800
181900
182000
182100
182200
182300
182400
182500
182600
182700
Row 182774: Invalid URI or not a track URI.
182800
182900
Row 182917: Invalid URI or not a track URI.
183000
183100
183200
183300
183400
183500
183600
183700
183800
183900
184000
184100
184200
184300
184400
184500
184600
184700
184800
184900
185000
185100
185200
185300
185400
185500
Row 185526: Invalid URI or not a track URI.
185600
185700
185800
185900
186000
186100
186200
186300
186400
186500
186600
186700
186800
186900
187000
187100
187200
187300
187400
187500
187600
187700
187800
187900


In [6]:
labelledkaggle = labelledkaggle.loc[:, ~labelledkaggle.columns.str.contains('^Unnamed')]

In [7]:

# Function to check if a given text is in English
def remove_non_english_lyrics(text):
    if isinstance(text, str):  # Check if text is a string
        try:
            lang = detect(text)
            if lang == 'en':
                return text
            else:
                return ''
        except:
            pass
    return text


labelledkaggle['lyrics'] = labelledkaggle['lyrics'].apply(remove_non_english_lyrics)
